In [2]:
pip install scx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 55.9 MB/s eta 0:00:00


In [3]:
from scx.optimize import Model

In [4]:
transport = [
    {
        'origin': 'U.S. Ainbolt',
        'destination': 'Factory 1',
        'distance': 120,
        'cost per mile':0.05,
        'unit cost':2.5
    },
    {
        'origin': 'U.S. Ainbolt',
        'destination': 'Factory 2',
        'distance': 380,
        'cost per mile':0.05,
        'unit cost':2.5
    },
    {
        'origin': 'U.S. Ainbolt',
        'destination': 'Factory 3',
        'distance': 806,
        'cost per mile':0.05,
        'unit cost':2.5
    },
    {
        'origin': 'Der Bolt, Thun',
        'destination': 'Factory 1',
        'distance': 465,
        'cost per mile':0.05,
        'unit cost':3.2
    },
    {
        'origin': 'Der Bolt, Thun',
        'destination': 'Factory 2',
        'distance': 717,
        'cost per mile':0.05,
        'unit cost':3.2
    },
    {
        'origin': 'Der Bolt, Thun',
        'destination': 'Factory 3',
        'distance': 853,
        'cost per mile':0.05,
        'unit cost':3.2
    },
    {
        'origin': 'Li Tningbolt',
        'destination': 'Factory 1',
        'distance': 553,
        'cost per mile':0.05,
        'unit cost':1.9
    },
    {
        'origin': 'Li Tningbolt',
        'destination': 'Factory 2',
        'distance': 572,
        'cost per mile':0.05,
        'unit cost':1.9
    },
    {
        'origin': 'Li Tningbolt',
        'destination': 'Factory 3',
        'distance': 830,
        'cost per mile':0.05,
        'unit cost':1.9
    },
    {
        'origin': 'M. Bolton',
        'destination': 'Factory 1',
        'distance': 915,
        'cost per mile':0.05,
        'unit cost':2.8,
    },
    {
        'origin': 'M. Bolton',
        'destination': 'Factory 2',
        'distance': 406,
        'cost per mile':0.05,
        'unit cost':2.8,
    },
    {
        'origin': 'M. Bolton',
        'destination': 'Factory 3',
        'distance': 564,
        'cost per mile':0.05,
        'unit cost':2.8,
    }
]

demand = [
    {
        'name':'Factory 1',
        'demand':9500
    },
    {
        'name':'Factory 2',
        'demand':6500
    },
    {
        'name':'Factory 3',
        'demand':17000
    }
]

supply = [
    {
        'name':'U.S. Ainbolt',
        'capacity':9500
    },
    {
        'name':'Der Bolt, Thun',
        'capacity':10000
    },
    {
        'name':'Li Tningbolt',
        'capacity':7000
    },
    {
        'name':'M. Bolton',
        'capacity':9000
    }
]

In [6]:
for t in transport:
  #Create the decision variables
  t['amt'] = Model.variable(name=f"{t['origin']}||{t['destination']}__amt", lowBound=0)
  #Calculate the total unit cost (transport + purchase)
  t['cost'] = t['distance']*t['cost per mile'] + t['unit cost']

In [7]:
print(transport[0])

{'origin': 'U.S. Ainbolt', 'destination': 'Factory 1', 'distance': 120, 'cost per mile': 0.05, 'unit cost': 2.5, 'amt': U.S._Ainbolt||Factory_1__amt, 'cost': 8.5}


In [8]:
#Initiate the model
my_model = Model(name = 'Locky Locke Inc', sense = 'minimize')

#Create the objective function
my_model.add_objective(
    fn = Model.sum([t['amt']*t['cost'] for t in transport])
)

#Add the constraints
##Demand constraint
for d in demand:
  my_model.add_constraint(
      name = f"{d['name']}__demand",
      fn = Model.sum([t['amt'] for t in transport if t['destination'] == d['name']]) >= d['demand']
  )

##Supply constraint
for s in supply:
  my_model.add_constraint(
      name = f"{s['name']}__capacity",
      fn = Model.sum([t['amt'] for t in transport if t['origin'] == s['name']]) <= s['capacity']
)

#Solve the model
my_model.solve()

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [9]:
my_model.show_formulation()

Locky_Locke_Inc:
MINIMIZE
26.45*Der_Bolt,_Thun||Factory_1__amt + 39.050000000000004*Der_Bolt,_Thun||Factory_2__amt + 45.85000000000001*Der_Bolt,_Thun||Factory_3__amt + 29.55*Li_Tningbolt||Factory_1__amt + 30.5*Li_Tningbolt||Factory_2__amt + 43.4*Li_Tningbolt||Factory_3__amt + 48.55*M._Bolton||Factory_1__amt + 23.1*M._Bolton||Factory_2__amt + 31.000000000000004*M._Bolton||Factory_3__amt + 8.5*U.S._Ainbolt||Factory_1__amt + 21.5*U.S._Ainbolt||Factory_2__amt + 42.800000000000004*U.S._Ainbolt||Factory_3__amt + 0.0
SUBJECT TO
Factory_1__demand: Der_Bolt,_Thun||Factory_1__amt
 + Li_Tningbolt||Factory_1__amt + M._Bolton||Factory_1__amt
 + U.S._Ainbolt||Factory_1__amt >= 9500

Factory_2__demand: Der_Bolt,_Thun||Factory_2__amt
 + Li_Tningbolt||Factory_2__amt + M._Bolton||Factory_2__amt
 + U.S._Ainbolt||Factory_2__amt >= 6500

Factory_3__demand: Der_Bolt,_Thun||Factory_3__amt
 + Li_Tningbolt||Factory_3__amt + M._Bolton||Factory_3__amt
 + U.S._Ainbolt||Factory_3__amt >= 17000

U.S._Ainbolt__capac

In [10]:
my_model.show_outputs()

{'objective': 923575.0,
 'status': 'Optimal',
 'variables': {'Der_Bolt,_Thun||Factory_1__amt': 0.0,
               'Der_Bolt,_Thun||Factory_2__amt': 0.0,
               'Der_Bolt,_Thun||Factory_3__amt': 7500.0,
               'Li_Tningbolt||Factory_1__amt': 0.0,
               'Li_Tningbolt||Factory_2__amt': 6500.0,
               'Li_Tningbolt||Factory_3__amt': 500.0,
               'M._Bolton||Factory_1__amt': 0.0,
               'M._Bolton||Factory_2__amt': 0.0,
               'M._Bolton||Factory_3__amt': 9000.0,
               'U.S._Ainbolt||Factory_1__amt': 9500.0,
               'U.S._Ainbolt||Factory_2__amt': 0.0,
               'U.S._Ainbolt||Factory_3__amt': 0.0}}
